In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
import pickle

# Specify the path to your pickle file
file_path1 = 'D:\Backend-Algorithm-main\Algorithm\EEG\EEG_schizophrenia\example#1\samples\Healthy.pkl'
file_path2 = 'D:\Backend-Algorithm-main\Algorithm\EEG\EEG_schizophrenia\example#1\samples\Schizopherina.pkl'



with open(file_path1, 'rb') as f:
    # Load the contents of the pickle file
    data_1 = pickle.load(f)


with open(file_path2, 'rb') as f:
    # Load the contents of the pickle file
    data_2 = pickle.load(f)


<>:4: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:4: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16064\4002451560.py:4: SyntaxWarning: invalid escape sequence '\B'
  file_path1 = 'D:\Backend-Algorithm-main\Algorithm\EEG\EEG_schizophrenia\example#1\samples\Healthy.pkl'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16064\4002451560.py:5: SyntaxWarning: invalid escape sequence '\B'
  file_path2 = 'D:\Backend-Algorithm-main\Algorithm\EEG\EEG_schizophrenia\example#1\samples\Schizopherina.pkl'


In [3]:

# Split each dataset into train and test sets
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    [data[0] for data in data_1], [data[1] for data in data_1], test_size=0.2, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    [data[0] for data in data_2], [data[1] for data in data_2], test_size=0.2, random_state=42)

# Combine training data and test data from both datasets
X_train = np.concatenate((X_train_1, X_train_2))
y_train = np.concatenate((y_train_1, y_train_2))
X_test = np.concatenate((X_test_1, X_test_2))
y_test = np.concatenate((y_test_1, y_test_2))

# Normalize the features
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)



# Split the test data into validation and final test sets
X_val, X_final_test, y_val, y_final_test = train_test_split(X_test_tensor, y_test_tensor, test_size=0.5, random_state=42)


In [4]:

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = NeuralNetwork(X_train_tensor.shape[1])

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model using validation set
num_epochs = 100
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on the validation set
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)
        predicted_val_classes = (val_outputs > 0.5).float()
        val_accuracy = (predicted_val_classes == y_val).float().mean()

    # Print loss and accuracy after every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy.item():.4f}')

# Evaluate the model on the final testing set
with torch.no_grad():
    final_test_outputs = model(X_final_test)
    final_test_loss = criterion(final_test_outputs, y_final_test)
    predicted_final_test_classes = (final_test_outputs > 0.5).float()
    final_test_accuracy = (predicted_final_test_classes == y_final_test).float().mean()

print("Final Test Loss:", final_test_loss.item())
print("Final Test Accuracy:", final_test_accuracy.item())


Epoch [1/100], Validation Loss: 0.6932, Validation Accuracy: 0.5412
Epoch [2/100], Validation Loss: 0.6906, Validation Accuracy: 0.5647
Epoch [3/100], Validation Loss: 0.6890, Validation Accuracy: 0.5647
Epoch [4/100], Validation Loss: 0.6881, Validation Accuracy: 0.5647
Epoch [5/100], Validation Loss: 0.6872, Validation Accuracy: 0.5647
Epoch [6/100], Validation Loss: 0.6863, Validation Accuracy: 0.5647
Epoch [7/100], Validation Loss: 0.6854, Validation Accuracy: 0.5647
Epoch [8/100], Validation Loss: 0.6846, Validation Accuracy: 0.5647
Epoch [9/100], Validation Loss: 0.6835, Validation Accuracy: 0.5647
Epoch [10/100], Validation Loss: 0.6822, Validation Accuracy: 0.5647
Epoch [11/100], Validation Loss: 0.6807, Validation Accuracy: 0.5647
Epoch [12/100], Validation Loss: 0.6789, Validation Accuracy: 0.5647
Epoch [13/100], Validation Loss: 0.6774, Validation Accuracy: 0.5647
Epoch [14/100], Validation Loss: 0.6758, Validation Accuracy: 0.5647
Epoch [15/100], Validation Loss: 0.6738, Va

In [5]:
torch.save(model, 'Best_Model.pth')